In [1]:
import pandas as pd

# Load the CSV file into a DataFrame
csv_file_8_16 = '/Users/adrianahernandezgonzalez/LabNotebook/11-24/states/dataDistances/subsampling_partial_ahCav12_8_16.csv'
df_8_16 = pd.read_csv(csv_file_8_16)

csv_file_32_64 = '/Users/adrianahernandezgonzalez/LabNotebook/11-24/states/dataDistances/subsampling_partial_ahCav12_32_64.csv'
df_32_64 = pd.read_csv(csv_file_32_64)

csv_file_256_512 = '/Users/adrianahernandezgonzalez/LabNotebook/11-24/states/dataDistances/subsampling_partial_ahCav12_256_512.csv'
df_256_512 = pd.read_csv(csv_file_256_512)

# Show the first few rows to confirm the data is loaded correctly
#print(df.head())

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [34]:
import pandas as pd

# Define function to calculate up/down probability for each VSD
def calculate_up_probability(row):
    # VSDI calculation
    vsdi_ratio = row["CA_ARG126_CA-PHE59_CA"] / row["CA_ARG135_CA-PHE59_CA"]
    vsdi_up_probability = vsdi_ratio if vsdi_ratio <= 1 else 1 / vsdi_ratio

    # VSDII calculation
    vsdii_ratio = row["CA_ARG509_CA-PHE456_CA"] / row["CA_LYS518_CA-PHE456_CA"]
    vsdii_up_probability = vsdii_ratio if vsdii_ratio <= 1 else 1 / vsdii_ratio

    # VSDIII calculation
    vsdiii_ratio = row["CA_LYS904_CA-PHE829_CA"] / row["CA_ARG916_CA-PHE829_CA"]
    vsdiii_up_probability = vsdiii_ratio if vsdiii_ratio <= 1 else 1 / vsdiii_ratio

    # VSDIV calculation
    vsdiv_ratio = row["CA_ARG1266_CA-PHE1167_CA"] / row["CA_LYS1275_CA-PHE1167_CA"]
    vsdiv_up_probability = vsdiv_ratio if vsdiv_ratio <= 1 else 1 / vsdiv_ratio

    return pd.Series([vsdi_up_probability, vsdii_up_probability, vsdiii_up_probability, vsdiv_up_probability])

# Apply function to each row in the dataframe and create a new dataframe for probabilities
up_prob_df = df_8_16[["pdb_file"]].copy()
up_prob_df[["VSDI_up_prob", "VSDII_up_prob", "VSDIII_up_prob", "VSDIV_up_prob"]] = df_8_16.apply(calculate_up_probability, axis=1)

# Display the result
print(up_prob_df)


                      pdb_file  VSDI_up_prob  VSDII_up_prob  VSDIII_up_prob  \
0     model_2_ptm_r3_seed1.pdb      0.865023       0.945442        0.876432   
1     model_3_ptm_r4_seed7.pdb      0.951868       0.819277        0.965453   
2     model_4_ptm_r1_seed6.pdb      0.866667       0.911444        0.894215   
3     model_5_ptm_r6_seed0.pdb      0.910159       0.835410        0.918101   
4    model_5_ptm_r11_seed2.pdb      0.941595       0.917798        0.998120   
..                         ...           ...            ...             ...   
645   model_4_ptm_r2_seed3.pdb      0.967765       0.894993        0.909947   
646   model_5_ptm_r5_seed5.pdb      0.947115       0.866271        0.980344   
647   model_3_ptm_r9_seed9.pdb      0.996302       0.850308        0.957291   
648  model_1_ptm_r12_seed2.pdb      0.923132       0.909942        0.986453   
649  model_4_ptm_r11_seed9.pdb      0.941009       0.836054        0.979939   

     VSDIV_up_prob  
0         0.839875  
1        

In [35]:
import matplotlib.pyplot as plt

# Plotting the distributions for each VSD in a semi-transparent style
plt.figure(figsize=(10, 6))

# Plot distributions with semi-transparency for each VSD
plt.hist(up_prob_df["VSDI_up_prob"], bins=10, alpha=0.5, label="VSDI", density=True)
plt.hist(up_prob_df["VSDII_up_prob"], bins=10, alpha=0.5, label="VSDII", density=True)
plt.hist(up_prob_df["VSDIII_up_prob"], bins=10, alpha=0.5, label="VSDIII", density=True)
plt.hist(up_prob_df["VSDIV_up_prob"], bins=10, alpha=0.5, label="VSDIV", density=True)

# Adding labels and title
plt.xlabel("Up Probability")
plt.ylabel("Density")
plt.title("Distribution of Up Probabilities for Each VSD MSA32-64")
plt.legend()

# Show the plot
plt.show()


In [36]:
def find_most_probable_down_models(df):
    """
    Orders all models by the lowest total up probabilities across all VSDs.

    Parameters:
    - df (DataFrame): The dataframe containing the model up probabilities.

    Returns:
    - DataFrame: A dataframe of all models ordered by total up probabilities.
    """
    # Calculate the sum of up probabilities for each model across all VSDs
    df['total_up_probability'] = df[['VSDI_up_prob', 'VSDII_up_prob', 'VSDIII_up_prob', 'VSDIV_up_prob']].sum(axis=1)
    
    # Sort the dataframe by the total_up_probability in ascending order
    ordered_models = df.sort_values(by='total_up_probability').reset_index(drop=True)
    
    # Drop the 'total_up_probability' column after sorting
    ordered_models = ordered_models.drop(columns=['total_up_probability'])
    
    return ordered_models


In [37]:
ordered_models = find_most_probable_down_models(up_prob_df)
ordered_models.iloc[600:651]

pdb_file  VSDI_up_prob  VSDII_up_prob  VSDIII_up_prob  \
600  model_5_ptm_r12_seed0.pdb      0.973130       0.868147        0.972705   
601   model_5_ptm_r5_seed5.pdb      0.947115       0.866271        0.980344   
602   model_1_ptm_r7_seed7.pdb      0.969697       0.817154        0.976658   
603   model_1_ptm_r9_seed6.pdb      0.870854       0.846903        0.981120   
604  model_2_ptm_r12_seed7.pdb      0.970717       0.913352        0.969438   
605   model_4_ptm_r5_seed0.pdb      0.926761       0.877339        0.994417   
606  model_5_ptm_r12_seed5.pdb      0.938703       0.894035        0.984117   
607   model_1_ptm_r6_seed8.pdb      0.995611       0.825789        0.968637   
608   model_2_ptm_r9_seed6.pdb      0.973094       0.864997        0.983374   
609  model_5_ptm_r12_seed3.pdb      0.940925       0.898208        0.998737   
610  model_5_ptm_r10_seed2.pdb      0.946121       0.923242        0.976294   
611  model_3_ptm_r11_seed4.pdb      0.949091       0.875785        0.955855   
612  model_5_ptm_r11_seed5.pdb      0.934844       0.894150        0.988208   
613  model_3_ptm_r12_seed4.pdb      0.949531       0.871148        0.966952   
614   model_5_ptm_r6_seed2.pdb      0.954480       0.954215        0.939104   
615   model_5_ptm_r9_seed7.pdb      0.981631       0.938182        0.938776   
616  model_2_ptm_r11_seed6.pdb      0.966767       0.894401        0.977231   
617   model_5_ptm_r6_seed3.pdb      0.946023       0.926429        0.979730   
618   model_5_ptm_r9_seed3.pdb      0.940014       0.946598        0.963781   
619   model_2_ptm_r7_seed4.pdb      0.964975       0.833903        0.989415   
620   model_4_ptm_r8_seed4.pdb      0.933902       0.885097        0.979012   
621  model_2_ptm_r12_seed1.pdb      0.960000       0.858383        0.949080   
622  model_1_ptm_r12_seed1.pdb      0.905444       0.992407        0.910226   
623  model_2_ptm_r11_seed1.pdb      0.947514       0.845127        0.946224   
624   model_5_ptm_r7_seed3.pdb      0.968908       0.906073        0.983760   
625   model_5_ptm_r8_seed3.pdb      0.942019       0.939942        0.980564   
626   model_3_ptm_r9_seed4.pdb      0.954578       0.894101        0.970260   
627  model_1_ptm_r12_seed2.pdb      0.923132       0.909942        0.986453   
628   model_5_ptm_r8_seed2.pdb      0.933048       0.935065        0.998118   
629  model_2_ptm_r12_seed4.pdb      0.981777       0.841522        0.970012   
630  model_5_ptm_r11_seed2.pdb      0.941595       0.917798        0.998120   
631   model_2_ptm_r4_seed4.pdb      0.975989       0.885559        0.976529   
632   model_2_ptm_r9_seed9.pdb      0.992943       0.927222        0.989335   
633  model_5_ptm_r11_seed3.pdb      0.942734       0.924964        0.996224   
634   model_4_ptm_r6_seed3.pdb      0.935889       0.850658        0.998767   
635   model_4_ptm_r2_seed3.pdb      0.967765       0.894993        0.909947   
636  model_2_ptm_r12_seed6.pdb      0.947524       0.954074        0.981663   
637   model_5_ptm_r7_seed2.pdb      0.947939       0.960613        0.985037   
638   model_5_ptm_r4_seed2.pdb      0.951694       0.977455        0.989487   
639  model_3_ptm_r10_seed4.pdb      0.975735       0.896113        0.993146   
640  model_1_ptm_r11_seed1.pdb      0.917271       0.999241        0.924596   
641  model_5_ptm_r12_seed2.pdb      0.934659       0.944365        0.975171   
642   model_4_ptm_r7_seed4.pdb      0.953538       0.876296        0.996226   
643  model_3_ptm_r12_seed9.pdb      0.983187       0.924177        0.969622   
644   model_5_ptm_r3_seed2.pdb      0.947406       0.947939        1.000000   
645   model_4_ptm_r3_seed2.pdb      0.922812       0.919255        0.971687   
646  model_2_ptm_r11_seed2.pdb      0.993304       0.855556        0.998777   
647  model_5_ptm_r10_seed3.pdb      0.957493       0.954512        0.996226   
648   model_4_ptm_r5_seed2.pdb      0.969587       0.849699        0.981074   
649   model_4_ptm_r4_seed2.pdb      0.973202       0.896412        0.964

In [6]:
#quality_df = pd.read_csv("/Users/adrianahernandezgonzalez/Documents/YarovLab/repositories/stateAnalysis/ModelQualityAssesment/model_quality_metrics_16.csv")
#quality_df

In [7]:
#merged_df = ordered_models.merge(quality_df, left_on="pdb_file", right_on="Filename", how="left")
#merged_df

In [38]:
import pandas as pd

def organize_by_distance(df, distance_column):
    """
    Organizes the dataframe by a specified distance column in ascending order.

    Parameters:
    - df (DataFrame): The dataframe to organize.
    - distance_column (str): The column name representing the distance to sort by.

    Returns:
    - DataFrame: The dataframe ordered by the specified distance column.
    """
    # Sort the dataframe by the specified distance column in ascending order
    organized_df = df.sort_values(by=distance_column).reset_index(drop=True)
    return organized_df

# Example usage:
organized_df = organize_by_distance(df_32_64, "CA_ARG509_CA-PHE456_CA")
print(organized_df)


                     pdb_file  shortest_ARG126-GLU52  shortest_ARG126-PHE59  \
0    model_1_ptm_r0_seed9.pdb                    3.4                   14.0   
1    model_1_ptm_r0_seed1.pdb                    2.6                   13.9   
2    model_2_ptm_r0_seed9.pdb                    3.3                   13.7   
3    model_1_ptm_r0_seed2.pdb                    2.2                   12.9   
4    model_3_ptm_r2_seed4.pdb                    2.6                   12.8   
..                        ...                    ...                    ...   
645  model_3_ptm_r1_seed1.pdb                    4.2                   13.7   
646  model_1_ptm_r0_seed8.pdb                    3.4                   15.0   
647  model_5_ptm_r0_seed9.pdb                    2.2                   12.4   
648  model_1_ptm_r1_seed3.pdb                    4.0                   15.4   
649  model_1_ptm_r0_seed4.pdb                    2.9                   14.0   

     shortest_ARG126-GLU62  shortest_ARG129-GLU52  

In [43]:
organized_df.iloc[1:31]#["CA_ARG509_CA-PHE456_CA"]

pdb_file  shortest_ARG126-GLU52  shortest_ARG126-PHE59  \
1    model_1_ptm_r0_seed1.pdb                    2.6                   13.9   
2    model_2_ptm_r0_seed9.pdb                    3.3                   13.7   
3    model_1_ptm_r0_seed2.pdb                    2.2                   12.9   
4    model_3_ptm_r2_seed4.pdb                    2.6                   12.8   
5    model_5_ptm_r1_seed0.pdb                    2.1                   11.9   
6    model_2_ptm_r0_seed8.pdb                    2.6                   14.5   
7    model_1_ptm_r0_seed0.pdb                    3.5                   14.7   
8    model_5_ptm_r0_seed0.pdb                    1.0                   11.6   
9   model_5_ptm_r12_seed7.pdb                    3.5                   13.7   
10   model_1_ptm_r0_seed3.pdb                    2.8                   14.1   
11   model_3_ptm_r0_seed1.pdb                    2.7                   13.9   
12   model_2_ptm_r0_seed2.pdb                    3.3                   10.0   
13   model_3_ptm_r0_seed8.pdb                    3.9                   14.5   
14   model_1_ptm_r0_seed5.pdb                    1.6                   12.2   
15   model_2_ptm_r1_seed1.pdb                    3.5                   14.4   
16   model_1_ptm_r2_seed2.pdb                    4.0                   14.9   
17   model_2_ptm_r1_seed8.pdb                    2.7                   13.5   
18   model_2_ptm_r0_seed3.pdb                    2.4                   13.4   
19   model_5_ptm_r1_seed4.pdb                    2.4                   13.2   
20   model_4_ptm_r0_seed1.pdb                    1.7                   13.0   
21   model_5_ptm_r0_seed4.pdb                    1.2                   11.8   
22   model_1_ptm_r1_seed9.pdb                    3.6                   15.1   
23   model_2_ptm_r0_seed7.pdb                    2.8                   14.4   
24   model_3_ptm_r1_seed5.pdb                    1.2                   12.1   
25   model_4_ptm_r1_seed4.pdb                    4.5                   15.2   
26   model_5_ptm_r0_seed1.pdb                    1.5                   13.0   
27   model_2_ptm_r3_seed9.pdb                    3.4                   14.4   
28   model_5_ptm_r2_seed8.pdb                    2.5                   12.3   
29   model_2_ptm_r1_seed0.pdb                    3.8                   13.6   
30   model_1_ptm_r3_seed2.pdb                    2.8                   13.7   

    shortest_ARG126-GLU62  shortest_ARG129-GLU52  shortest_ARG129-PHE59  \
1                    18.9                    5.1                    6.8   
2                    18.5                    7.1                    6.9   
3                    17.9                    5.9                    6.1   
4                    17.4                    4.8                    8.2   
5                    16.3                    5.5                    7.6   
6                    18.6                    5.1                    7.9   
7                    19.0                    5.2                    8.8   
8                    17.1                    6.6                    6.2   
9                    17.9                    4.9                    8.9   
10                   18.7                    5.1                    6.9   
11                   18.8                    5.3                    7.2   
12                   14.1                   10.2                    5.4   
13                   19.9                    4.8                    6.5   
14                   16.9                    6.1                    6.3   
15                   19.7                    5.0                    6.9   
16                   19.1                    4.3                    9.7   
17                   18.0                    6.2                    6.0   
18                   18.2                    8.4                    6.3   
19                   17.7                    6.0                    8.0   
20                   18.0                    6.6                    5.8

In [40]:
df_32_64

pdb_file  shortest_ARG126-GLU52  shortest_ARG126-PHE59  \
0     model_2_ptm_r3_seed1.pdb                    4.0                   14.7   
1     model_3_ptm_r4_seed7.pdb                    4.0                   14.6   
2     model_4_ptm_r1_seed6.pdb                    3.6                   13.6   
3     model_5_ptm_r6_seed0.pdb                    3.4                   13.4   
4    model_5_ptm_r11_seed2.pdb                    3.1                   13.3   
..                         ...                    ...                    ...   
645   model_4_ptm_r2_seed3.pdb                    5.0                   15.3   
646   model_5_ptm_r5_seed5.pdb                    2.7                   13.7   
647   model_3_ptm_r9_seed9.pdb                    3.1                   13.7   
648  model_1_ptm_r12_seed2.pdb                    3.2                   13.9   
649  model_4_ptm_r11_seed9.pdb                    4.5                   14.8   

     shortest_ARG126-GLU62  shortest_ARG129-GLU52  shortest_ARG129-PHE59  \
0                     19.3                    4.7                    7.9   
1                     18.8                    4.5                    9.9   
2                     18.2                    5.8                    6.5   
3                     17.5                    5.8                    7.9   
4                     17.7                    3.7                    8.7   
..                     ...                    ...                    ...   
645                   19.5                    4.9                    9.4   
646                   18.2                    4.8                    9.5   
647                   18.3                    4.5                    9.3   
648                   18.2                    4.4                    8.9   
649                   18.9                    4.5                    9.7   

     shortest_ARG129-GLU62  shortest_ARG132-GLU52  shortest_ARG132-PHE59  \
0                     12.3                   11.6                    2.7   
1                     13.5                    9.6                    3.1   
2                     10.6                   12.3                    3.1   
3                     11.7                   11.6                    2.8   
4                     12.3                    8.7                    2.9   
..                     ...                    ...                    ...   
645                   13.3                    9.5                    3.2   
646                   13.3                   10.0                    3.2   
647                   13.3                    9.5                    2.8   
648                   13.0                    9.9                    2.2   
649                   13.5                    9.1                    3.2   

     shortest_ARG132-GLU62  ...  CA_GLU252_CA-GLU1353_CA  \
0                      5.2  ...                     7.23   
1                      6.9  ...                     7.08   
2                      4.2  ...                     7.66   
3                      5.0  ...                     6.87   
4                      6.2  ...                     6.96   
..                     ...  ...                      ...   
645                    6.9  ...                     7.19   
646                    6.6  ...                     7.11   
647                    6.6  ...                     7.36   
648                    6.4  ...                     7.00   
649                    7.3  ...                     7.76   

     CA_GLU595_CA-GLU1024_CA  CA_GLU595_CA-GLU1353_CA  \
0                       6.63                     9.84   
1                       6.65                     9.63   
2                       7.22                     9.84   
3                       6.59                     9.26   
4                       6.58                     9.19   
..                       ...                      ...   
645                     6.83                     9.53   
646                     6.50                     9.29   
6

In [41]:
import pandas as pd
import matplotlib.pyplot as plt

def plot_multiple_distances_histogram(df, distance_dict, bins=10, alpha=0.5):
    """
    Plots histograms of distances for specified columns in the dataframe with given aliases.

    Parameters:
    - df (DataFrame): The dataframe containing the distances.
    - distance_dict (dict): A dictionary where keys are aliases, and values are column names.
    - bins (int): Number of bins for the histogram (default is 10).
    - alpha (float): Transparency level for the histogram plots (default is 0.5).
    """
    plt.figure(figsize=(10, 6))
    
    # Loop through each item in the dictionary and plot the histogram for each distance column
    for alias, column_name in distance_dict.items():
        if column_name in df.columns:
            plt.hist(df[column_name], bins=bins, alpha=alpha, label=alias, edgecolor='black')
        else:
            print(f"Warning: Column '{column_name}' not found in the dataframe.")

    # Set plot labels and title
    plt.xlabel("Distance")
    plt.ylabel("Frequency")
    plt.title("Histogram of Distances for Specified Columns")
    plt.legend()
    plt.show()

# Example usage:
# Assuming df is your DataFrame with distance columns
distance_dict = {
    "ARG126-GLU52": "shortest_ARG126-GLU52",
    "ARG126-PHE59": "shortest_ARG126-PHE59",
    "ARG129-GLU52": "shortest_ARG129-GLU52"
}

# Plot the histograms
# plot_multiple_distances_histogram(df, distance_dict)
plot_multiple_distances_histogram(df_32_64, distance_dict, bins=10, alpha=0.5)